In [2]:
!git lfs install
!git clone https://huggingface.co/spaces/Plachta/VALL-E-X

Git LFS initialized.
Cloning into 'VALL-E-X'...
remote: Enumerating objects: 295, done.
remote: Total 295 (delta 0), reused 0 (delta 0), pack-reused 295 (from 1)
Receiving objects: 100% (295/295), 1.34 MiB | 1.92 MiB/s, done.
Resolving deltas: 100% (126/126), done.
Filtering content: 100% (46/46), 1.38 GiB | 30.97 MiB/s, done.


In [3]:
%cd /kaggle/working/VALL-E-X

/kaggle/working/VALL-E-X


In [4]:
!pip install cmake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 73.1 MB/s eta 0:00:00:00:0100:01


In [5]:
!pip install -r requirements.txt
!pip install torchaudio==2.0.2 --no-cache-dir
!pip install pytorch-lightning==2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 35.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.2 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 39.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s e

## TEXT

In [6]:
text = "I'm sorry to hear that.  What is the project about?  Do you think you can get it done?"

In [7]:
output_wav = "/kaggle/working/new_test.wav"

## SEPARATE FILE

In [8]:
if len(text) > 150:
    print("Rejected, Text too long (should be less than 150 characters")

In [9]:
import torch

In [10]:
NUM_LAYERS = 12
NUM_HEAD = 16
N_DIM = 1024
PREFIX_MODE = 1
NUM_QUANTIZERS = 8
SAMPLE_RATE = 24000

In [11]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda", 0)

In [12]:
%cd VALL-E-X

[Errno 2] No such file or directory: 'VALL-E-X'
/kaggle/working/VALL-E-X


In [13]:
from models.vallex import VALLE

In [14]:
import platform
import os
import pathlib
if platform.system().lower() == 'windows':
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
else:
    temp = pathlib.WindowsPath
    pathlib.WindowsPath = pathlib.PosixPath
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [15]:
# VALL-E-X model
model = VALLE(
    N_DIM,
    NUM_HEAD,
    NUM_LAYERS,
    norm_first=True,
    add_prenet=False,
    prefix_mode=PREFIX_MODE,
    share_embedding=True,
    nar_scale_factor=1.0,
    prepend_bos=True,
    num_quantizers=NUM_QUANTIZERS,
).to(device)
checkpoint = torch.load("epoch-10.pt", map_location='cpu')
missing_keys, unexpected_keys = model.load_state_dict(
    checkpoint["model"], strict=True
)
del checkpoint
assert not missing_keys
model.eval()

VALLE(
  (ar_text_embedding): TokenEmbedding(
    (dropout): Dropout(p=0.0, inplace=False)
    (word_embeddings): Embedding(2048, 1024)
  )
  (nar_text_embedding): TokenEmbedding(
    (dropout): Dropout(p=0.0, inplace=False)
    (word_embeddings): Embedding(2048, 1024)
  )
  (ar_audio_embedding): TokenEmbedding(
    (dropout): Dropout(p=0.0, inplace=False)
    (word_embeddings): Embedding(1026, 1024)
  )
  (ar_text_prenet): Identity()
  (ar_audio_prenet): Identity()
  (ar_text_position): SinePositionalEmbedding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (ar_audio_position): SinePositionalEmbedding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (ar_decoder): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (linear1): Linear(in_features=1024, out_features=4096, bias=True)
   

In [16]:
audio_prompts = torch.zeros([1, 0, NUM_QUANTIZERS]).type(torch.int32).to(device)
text_prompts = torch.zeros([1, 0]).type(torch.int32)
lang_pr = 'en'
text_pr = ""
enroll_x_lens = 0
wav_pr, sr = None, None

In [17]:
langdropdown2token = {
    'English': "[EN]",
    '中文': "[ZH]",
    '日本語': "[JA]",
    'Mix': "",
}

In [18]:
lang2token = {
    'zh': "[ZH]",
    'ja': "[JA]",
    "en": "[EN]",
    'mix': "",
}

In [19]:
token2lang = {
    '[ZH]': "zh",
    '[JA]': "ja",
    "[EN]": "en",
    "": "mix"
}

In [20]:
import langid
langid.set_languages(['en', 'zh', 'ja'])

In [21]:
lang_token = lang2token[langid.classify(text)[0]]
lang = token2lang[lang_token]
text = text.replace("\n", "")
text = lang_token + text + lang_token

In [22]:
import logging

In [23]:
!pip install unidecode

In [24]:
!pip install cn2an

In [25]:
!pip install inflect

In [26]:
from utils.g2p import PhonemeBpeTokenizer
from data.collation import get_text_token_collater

In [27]:
text_tokenizer = PhonemeBpeTokenizer(tokenizer_path="./utils/g2p/bpe_69.json")
text_collater = get_text_token_collater()

In [28]:
!pip install eng_to_ipa

In [29]:
# tokenize text
logging.info(f"synthesize text: {text}")
phone_tokens, langs = text_tokenizer.tokenize(text=f"_{text}".strip())
text_tokens, text_tokens_lens = text_collater(
    [
        phone_tokens
    ]
)

In [30]:
accent = 'English'

In [31]:
text_tokens = torch.cat([text_prompts, text_tokens], dim=-1)
text_tokens_lens += enroll_x_lens
lang = lang if accent == "no-accent" else token2lang[langdropdown2token[accent]]
encoded_frames = model.inference(
    text_tokens.to(device),
    text_tokens_lens.to(device),
    audio_prompts,
    enroll_x_lens=enroll_x_lens,
    top_k=-100,
    temperature=1,
    prompt_language=lang_pr,
    text_language=langs if accent == "no-accent" else lang,
)

VALL-E EOS [0 -> 395]
Current memory used: 2287.46 MB


In [32]:
!pip install vocos

In [33]:
from vocos import Vocos

In [34]:
vocos = Vocos.from_pretrained('charactr/vocos-encodec-24khz').to(device)

config.yaml:   0%|          | 0.00/503 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/40.4M [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:01<00:00, 80.1MB/s]


In [35]:
# Decode with Vocos
frames = encoded_frames.permute(2,0,1)
features = vocos.codes_to_features(frames)
samples = vocos.decode(features, bandwidth_id=torch.tensor([2], device=device))


In [36]:
audio = (24000, samples.squeeze(0).cpu().numpy())

## Save AUDIO

In [37]:
import warnings
import numpy as np

In [38]:
def convert_to_16_bit_wav(data):
    # Based on: https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.write.html
    warning = "Trying to convert audio automatically from {} to 16-bit int format."
    if data.dtype in [np.float64, np.float32, np.float16]:
        warnings.warn(warning.format(data.dtype))
        data = data / np.abs(data).max()
        data = data * 32767
        data = data.astype(np.int16)
    elif data.dtype == np.int32:
        warnings.warn(warning.format(data.dtype))
        data = data / 65536
        data = data.astype(np.int16)
    elif data.dtype == np.int16:
        pass
    elif data.dtype == np.uint16:
        warnings.warn(warning.format(data.dtype))
        data = data - 32768
        data = data.astype(np.int16)
    elif data.dtype == np.uint8:
        warnings.warn(warning.format(data.dtype))
        data = data * 257 - 32768
        data = data.astype(np.int16)
    elif data.dtype == np.int8:
        warnings.warn(warning.format(data.dtype))
        data = data * 256
        data = data.astype(np.int16)
    else:
        raise ValueError(
            "Audio data cannot be converted automatically from "
            f"{data.dtype} to 16-bit int format."
        )
    return data

In [39]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")  # Ignore pydub warning if ffmpeg is not installed
    from pydub import AudioSegment

In [40]:
def audio_to_file(sample_rate, data, filename, format="wav"):
    if format == "wav":
        data = convert_to_16_bit_wav(data)
    audio = AudioSegment(
        data.tobytes(),
        frame_rate=sample_rate,
        sample_width=data.dtype.itemsize,
        channels=(1 if len(data.shape) == 1 else data.shape[1]),
    )
    file = audio.export(filename, format=format)
    file.close()  # type: ignore

In [41]:
format = "wav"

In [42]:
audio_to_file(
    audio[0], audio[1], output_wav, format=format
)

/tmp/ipykernel_23/2774573727.py:5: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
